In [3]:
import numpy as np
from collections import Counter
class ability:
    def __init__(self, consumption, product_time, pop, status, prices, wealth):
        self.consumption = np.array(consumption, dtype=float)
        self.product_time = product_time
class TradeBlock(ability):
    def __init__(self, consumption, pop, product_time, status, prices, wealth):
        """
        经济模拟系统中的有贸易区块类。
        """
        # 读取参数，状态和工作时间
        super().__init__(consumption, product_time, pop, status, prices, wealth)
        self.pop = np.array(pop, dtype=float) # 集体的库存
        self.status = np.array(status, dtype=float) # 集体的库存
        self.prices = np.array(prices, dtype=float) # 商品的市场价格
        self.wealth = np.array(wealth, dtype=float) # 各职业手中的货币
    def evolution(self, consumption):
        """
        经济演化过程。

        """
        # 计算每日需要消耗的货币量
        self.daily_cost = (self.consumption * self.prices).sum(axis = 1)
        # 计算每日需要消耗的商品
        self.daily_consumption = (self.consumption.T * self.pop).sum(axis = 1)

        # 计算各职业每日工作时长 = 日常消耗 / 生产单位商品所需时间,上限一天
        self.worktime = np.round((self.daily_cost / self.prices) * self.product_time,2)
        self.worktime[self.worktime>1] = 1

        # 计算各职业每日生产量 = 工作时长 / 生产单位商品所需时间
        self.production = self.pop * np.round(self.worktime / self.product_time,2)
        # 出售所得货币量 = 生产量 × 产品价格
        self.soldvalue = np.round(self.production * self.prices,2)
        # (待出售的)新状态 = 旧状态 + 产量
        self.status = self.status + (self.production)[1:]

        # 计算各职业新的财富状态
        earned = self.wealth[1:] + self.soldvalue[1:]
        self.wealth[0] = self.wealth[0] - sum(earned) + self.soldvalue[0]
        self.wealth[1:] = self.wealth[1:] + earned
        self.per_wealth = self.wealth / self.pop

        # 计算库存目标
        self.product_aim = np.array([20,10]) * sum(self.pop)
        # 计算各职业储蓄目标
        self.save_aim = 10 * (self.consumption * self.prices).sum(axis = 1)

        # # 计算新库存状态：旧库存 + 生产 - 消耗 + 贸易变化
        # self.status = self.status + production - self.consumption
test = TradeBlock(
[[0,2,2],[0,0,1],[0,1,0]],#consumption
[2,15,8],#pop
[1,0.5,0.2],#product time
[0,0],#status(storage)
[1,2,3],#prices
[0,0,0])#wealth

In [4]:
test1 = test.evolution([[0,2,2],[0,0,1],[0,1,0]])

In [5]:
test.pop

array([ 2., 15.,  8.])

In [6]:
test.daily_consumption

array([ 0., 12., 19.])

In [7]:
test.daily_cost

array([10.,  3.,  2.])

In [8]:
test.worktime

array([1.  , 0.75, 0.13])

In [89]:
test.production

array([ 2. , 22.5,  5.2])

In [90]:
test.wealth

array([-58.6,  45. ,  15.6])

In [92]:
test.product_aim

array([500., 250.])

In [93]:
test.save

array([100.,  30.,  20.])

In [94]:
test.per_wealth

array([-29.3 ,   3.  ,   1.95])